# Programa para testear el correcto funcionamiento del procesador de datos

Este programa sirve para comprobar que los resultados obtenidos a partir del programa "Procesador_base_datos" son correctos.

## Librerias

In [1]:
import os
import pandas as pd
import numpy as np

## Definición de las direcciones

Primero se ha de indicar el conjunto que se quiere revisar (Train, Test o Val) y el fichero de datos que se quiere verificar. Para ello hay que especificar lo siguiente:

In [19]:
#Indique que el conjunto que quiere revisar
Conjunto = "Train" 

#Indique el nombre del fichero en el que se encontraban los datos originales (direccion a partir de la carpeta del conjunto)
Nombre_fichero_original = "Mes2202-28022022/Nexus/t1_Nexus.csv" 

#Indique la fecha y la hora en la que se procesaron los datos
Fecha = "22_9_2022"
Hora = "8:44:44"

Lo siguiente que hacemos es verificar que las direcciones estén bien, de haber alguna incorrecta se comunicará a continuación a modo de seguridad antes de correr el programa.

In [20]:
#Definición de las direcciones 
current_path = os.getcwd()
path_original_data = current_path + "/Database/Raw_data" + "/"+ Conjunto +"/" + Nombre_fichero_original
path_processed_data = current_path + "/Database/Processed_data" + "/"+ Fecha +"/"+Hora+"/matriz_"+Conjunto +"_organizada.csv"
path_info_data = current_path + "/Database/Processed_data" + "/"+ Fecha +"/"+Hora+"/orden_"+Conjunto +".csv"

#Nos aseguramos de que las direcciones existan
assert os.path.exists(path_original_data), "Dirección " + str(path_original_data) + " de los datos originales errónea."
print("Datos originales encontrados." if os.path.exists(path_original_data) else "Dirección de los datos originales errónea.")

assert os.path.exists(path_processed_data), "Dirección " + str(path_processed_data) + " de los datos procesados errónea."
print("Datos procesados encontrados." if os.path.exists(path_processed_data) else "Dirección de los datos procesados errónea.")

assert os.path.exists(path_info_data), "Información de los datos errónea. revise que la direccion "+str(path_info_data)+" sea correcta."
print("Información de los datos encontrada." if os.path.exists(path_info_data) else "Información de los datos errónea.")


Datos originales encontrados.
Datos procesados encontrados.
Información de los datos encontrada.


## Obtención de las matrices

Una vez las direcciones se han verificado se procede a la extracción de los datos. Se muestra también una pequeña parte de cada una para verificar que todo está bien.

In [3]:
#Datos originales
datos_originales=(pd.read_csv(path_original_data, on_bad_lines='skip', header = None)).to_numpy().tolist()
print("\u25CF Cinco primeras filas de \033[1mdatos sin procesar\033[0m:\n" + str(datos_originales[:5]))

#Información de los datos
info_data=(pd.read_csv(path_info_data, on_bad_lines='skip', header = None)).to_numpy().tolist()
print("\u25CF Cinco primeras filas de \033[1minformación\033[0m:\n"+str(info_data[:5]))

#Datos procesados
datos_procesados=pd.read_csv(path_processed_data, on_bad_lines='skip')

matriz = datos_procesados.to_numpy()
columnas = (datos_procesados.columns).to_numpy().tolist()
print("\u25CF \033[1mMatriz procesada\033[0m")
print("En total hay " + str(len(columnas)) + " columnas, que representan distintos AP's (y la primer vacía que hace referencia al numero de muestra). Aquí te muestro las 5 primeras:\n"+str(columnas[:5]))
print("El resto de la matriz es de tamaño "+str(matriz.shape[0])+"x"+str(matriz.shape[1])+ ". Aquí un ejemplo de las primeras 5 filas y columnas:\n"+str(matriz[:5,:5]))


● Cinco primeras filas de datos sin procesar:
[[0, 'iPhone de Lau', '42:15:47:6a:eb:11', -29, '28-feb.-2022 07:28:13 p. m.', '40.64228569--3.15151077', 12813462447], [0, 'vodafoneEF28-5G', 'f8:08:4f:75:ef:2f', -71, '28-feb.-2022 07:28:13 p. m.', '40.64228569--3.15151077', 12805539412], [0, nan, '5a:30:d9:15:e2:fe', -73, '28-feb.-2022 07:28:13 p. m.', '40.64228569--3.15151077', 12813462569], [0, 'MOVISTAR_8E86', 'd8:fb:5e:86:8e:87', -67, '28-feb.-2022 07:28:13 p. m.', '40.64228569--3.15151077', 12813462539], [0, 'Orange-EC4F', 'd4:63:fe:4a:ec:51', -76, '28-feb.-2022 07:28:13 p. m.', '40.64228569--3.15151077', 12805539535]]
● Cinco primeras filas de información:
[['/home/laura/Adrian/Procesaor_dataset/Database/Raw_data/Train/Mes06-29062021/Nexus/t1_Nexus.csv', 128], ['/home/laura/Adrian/Procesaor_dataset/Database/Raw_data/Train/Mes06-29062021/Nexus/t2_Nexus.csv', 134], ['/home/laura/Adrian/Procesaor_dataset/Database/Raw_data/Train/Mes06-29062021/Nexus/t3_Nexus.csv', 107], ['/home/laura/A

## Localización de los datos iniciales

Para poder localizar el fichero dentro de la matriz lo primero será buscar la fila a partir de la cual se localizan los datos.

In [4]:
#Sacamos las listas con las direcciones y con las secuencias
direcciones = [item[0] for item in info_data]
secuencias = [item[1] for item in info_data]
#print(direcciones[:3])
#print(secuencias[:3])

#Localizamos la fila en la que se encuentran nuestros datos
iden = direcciones.index(path_original_data)
#print(iden)

fila_inicio = sum(secuencias[:iden])+(iden)
#print(fila_inicio)


## Comprobamos que los valores coincidan

Una vez se han llevado a cabo los pasos anteriores pasamos a verificar la matriz. Para ello observamos los datos riginales uno a uno, calculando la posición en la que se han de encontrar dentro de la matriz, y allí verificamos que el valor encontrado y el esperado coincidan.

In [6]:
id_fallo=[]
MAC_fallo=""
muestra_fallo=0
fallos_repeticion=0

for i,element in enumerate(datos_originales):    
    
    #Comprobamos que este la dirección MAC en la lista de columnas
    if(element[2] in columnas):
        
        col = columnas.index(element[2])
        fila=fila_inicio+element[0]
        #print(fila,col)
        if((element[0] == muestra_fallo) and (element[2] == MAC_fallo)):
            print("\u2192Fallo por repetición de MAC.")
            fallos_repeticion+=1
        
        if(element[3]!=matriz[fila][col]):
            print("\u25CF \033[1mAlgo ha fallado\033[0m")
            print("Fila: "+str(fila)+"  Columna: "+str(col))
            print("Mac: "+str(element[2])+", número de muestra: "+str(element[0]))
            print("Valor esperado: "+str(element[3])+  ", valor obtenido: "+ str(matriz[fila][col]))
            
            muestra_fallo = element[0]
            MAC_fallo = element[2]
                       
            if(element[2] not in id_fallo):
                id_fallo.append([element[2],i,fila,col])
    
    #Si no lo está sacamos el error
    else:
        print("\u25CF \033[1mFallo al encontrar la columna\033[0m")

                
#print(id_fallo)

● Algo ha fallado
Fila: 2154  Columna: 1001
Mac: fe:49:2d:cb:f9:12, número de muestra: 3
Valor esperado: -78, valor obtenido: -76.0
→Fallo por repetición de MAC.
● Algo ha fallado
Fila: 2155  Columna: 1001
Mac: fe:49:2d:cb:f9:12, número de muestra: 4
Valor esperado: -77, valor obtenido: -76.0
→Fallo por repetición de MAC.
● Algo ha fallado
Fila: 2156  Columna: 1001
Mac: fe:49:2d:cb:f9:12, número de muestra: 5
Valor esperado: -77, valor obtenido: -76.0
→Fallo por repetición de MAC.
● Algo ha fallado
Fila: 2157  Columna: 1001
Mac: fe:49:2d:cb:f9:12, número de muestra: 6
Valor esperado: -77, valor obtenido: -76.0
→Fallo por repetición de MAC.
● Algo ha fallado
Fila: 2158  Columna: 1001
Mac: fe:49:2d:cb:f9:12, número de muestra: 7
Valor esperado: -77, valor obtenido: -76.0
→Fallo por repetición de MAC.
● Algo ha fallado
Fila: 2159  Columna: 1001
Mac: fe:49:2d:cb:f9:12, número de muestra: 8
Valor esperado: -77, valor obtenido: -76.0
→Fallo por repetición de MAC.
● Algo ha fallado
Fila: 2160

## En caso de fallo

Por último en caso de que haya habido algún fallo "traducimos" dicho fallo para que sea facilmente localizable.

In [7]:
dic ={
    "1" : "A",
    "2" : "B",
    "3" : "C",
    "4" : "D",
    "5" : "E",
    "6" : "F",
    "7" : "G",
    "8" : "H",
    "9" : "I",
    "10" : "J",
    "11" : "K",
    "12" : "L",
    "13" : "M",
    "14" : "N",
    "15" : "O",
    "16" : "P",
    "17" : "Q",
    "18" : "R",
    "19" : "S",
    "20" : "T",
    "21" : "U",
    "22" : "V",
    "23" : "W",
    "24" : "X",
    "25" : "Y",
    "26" : "Z"
}

def Traductor_excel(fila, columna):
    
    base=26
    digitos=[]
    valor=columna+1
    resultado= ""
    
    continua = True
    
    while (continua==True):
        
        digitos.append((valor%base))
        valor = int(valor/base)
        
        if(valor <= base):
            digitos.append(valor)
            continua=False
    
    for digito in reversed(digitos):
        resultado += dic[str(digito)]
    
    resultado += str(fila+2)
    
    return resultado


In [21]:
print("Fallos encontrados: "+str(len(id_fallo))+", de los cuales " +str(fallos_repeticion)+ " han sido por repetición de MAC.")
for fallo in id_fallo:
    print("\u25CF \033[1mRevisar:\033[0m")
    print("\t \u2192 En los datos originales la fila " + str(fallo[1]+1)+" que se ha de corresponder con la MAC "+str(fallo[0]))
    print("\t \u2192 En la matriz procesada la casilla " + str(Traductor_excel(fallo[2], fallo[3])))
    

Fallos encontrados: 22, de los cuales 22 han sido por repetición de MAC.
● Revisar:
	 → En los datos originales la fila 139 que se ha de corresponder con la MAC fe:49:2d:cb:f9:12
	 → En la matriz procesada la casilla ALN2156
● Revisar:
	 → En los datos originales la fila 177 que se ha de corresponder con la MAC fe:49:2d:cb:f9:12
	 → En la matriz procesada la casilla ALN2157
● Revisar:
	 → En los datos originales la fila 206 que se ha de corresponder con la MAC fe:49:2d:cb:f9:12
	 → En la matriz procesada la casilla ALN2158
● Revisar:
	 → En los datos originales la fila 238 que se ha de corresponder con la MAC fe:49:2d:cb:f9:12
	 → En la matriz procesada la casilla ALN2159
● Revisar:
	 → En los datos originales la fila 269 que se ha de corresponder con la MAC fe:49:2d:cb:f9:12
	 → En la matriz procesada la casilla ALN2160
● Revisar:
	 → En los datos originales la fila 296 que se ha de corresponder con la MAC fe:49:2d:cb:f9:12
	 → En la matriz procesada la casilla ALN2161
● Revisar:
	 → 